In [1]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [2]:
# which_annotation = "contains_requirements"
# # get the total number of chunks that contain the annotation
# total = 0
# list_of_chunks = []
# for i, annotation in enumerate(annotations):
#     if annotation[which_annotation]:
#         total += 1
#         list_of_chunks.append(i)
# print(total)

In [3]:
from rfpretriever.RFPRetriever import RFPRetriever
import logging

logging.basicConfig(level=logging.INFO)

rfp = RFPRetriever(
    pdf_path='./rfpparser/RFP 2032H323 Amendment.pdf',
    verbose=True
)

INFO:root:PDFParser for ./rfpparser/RFP 2032H323 Amendment.pdf
INFO:root:Got 290 chunks
100%|██████████| 290/290 [02:20<00:00,  2.07it/s]


TypeError: cannot pickle 're.Match' object

In [5]:
chunks = [rfp.chunks()[i] for i in list_of_chunks]
chunks

[{'text': "2032H323R00065 – AMENDMENT 1   \n4 of 91 \n 2032H323R00065 – AMENDMENT 1, ON-DEMAND EXECUTIVE COACHING WITH \nASSESSMENTS & LEADERSHIP DEVELOPMENT RESOURCES  . 6 \nSECTION I - STATEMENT OF WORK (SOW)  . 8 \nSECTION II – CONTRACT ADMINISTRATION, TERMS AND CONDITIONS  . 14 \n1.0 AUTHORITY . 14 \nDTAR 1052.201-70 Contracting Officer's Representative (COR) Appointment and Authority (Apr \n2015) . 14 \n2.0 TYPE OF CONTRACT . 16 \n3.0 INVOICE SUBMISSION REQUIREMENTS . 16 \nDTAR 1052.232-7003 - Electronic Submission of Payment Requests (Apr 2015) . 16 \n4.0 CONTRACT CLAUSES . 18 \nFAR 52.212-5 - Contract Terms and Conditions Required to Implement Statutes or Executive \nOrders— Commercial Products and Commercial Services (Jun 2023) .",
  'metadata': {'page_no': '4',
   'file_type': 'application/pdf',
   'chunk_no': 12}},
 {'text': '(xi) Delivery of all reports and services will be made to: \n \nU.S. Department of the Treasury \nOffice of Treasury Executive Institute (TEI) \nATTN:  

In [6]:
PROMPT = """For the given excerpt from a RFP, list out the technical requirements.
Excerpt:
{excerpt}
Response:
"""
print(PROMPT)

For the given excerpt from a RFP, list out the technical requirements.
Excerpt:
{excerpt}
Response:



In [7]:
extracted_information = []
for chunk in chunks:
    response = llm.complete(PROMPT.format(excerpt=chunk['text'])).text
    extracted_information.append(response)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

In [8]:
extracted_information

['- On-demand executive coaching with assessments\n- Leadership development resources\n- Electronic submission of payment requests\n- Contract terms and conditions required to implement statutes or executive orders for commercial products and services',
 'Technical requirements:\n- Delivery of all reports and services to the specified address\n- Compliance with FAR Provision 52.212-1, Instructions to Offerors-Commercial Products and Commercial Services\n- Compliance with FAR Provision 52.212-2, Evaluation-Commercial Products and Commercial Services\n- Inclusion of completed copy of provision at 52.212-3, Offeror Representations and Certifications-Commercial Products and Commercial Services',
 '- 1:1 Coaching by ICF credentialed coaches\n- On-Demand Coaching with sessions occurring within one business day\n- Learning/Development Resources to support coachees in coaching areas\n- Assessment to be completed at start and end of engagements',
 'Technical requirements:\n- Development of a co

In [10]:
# export the extracted information to a file
with open('extracted_information.txt', 'w') as f:
    for item in extracted_information:
        f.write("%s\n" % item)

In [21]:
new_requirmeents = []
# A detailed description of the work that the government is hiring for. This includes a high-level description, 
# detailed deliverables, requirements that need to be met, and may also include certifications and qualifications. 
PROMPT2 = """For the given excerpt from a RFP, if present, extract actionable requirements that teh applicant needs to ensure in the application.
----------------
Example 1:
Excerpt:
The Contractor shall provide online subscription services with online access to the digital libraries via the website platform, 24 hours a day, 7 days a week, with customer support.
Response:
24/7 online access to digital libraries via the website platform with 24/7 customer support.
----------------
Example 2:
Engagement with Coaches 
  Total number of sessions completed 
  Total time spent in sessions 
  Average number of completed sessions and time in session per coachee 
  Most common topics discussed in coaching sessions (denoted by percentage)
Response:
None
Excerpt:
{excerpt}
Response:
"""

In [22]:
import tqdm
for chunk in tqdm.tqdm(chunks):
    response = llm.complete(PROMPT2.format(excerpt=chunk['text'])).text
    new_requirmeents.append(response)

  0%|          | 0/24 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
100%|██████████| 24/24 [00:34<00:00,  1.43s/it]


In [23]:
new_requirmeents = [x for x in new_requirmeents if x!='None']
new_requirmeents

['- Ensure compliance with DTAR 1052.232-7003 for electronic submission of payment requests.',
 '- Delivery of all reports and services to the U.S. Department of the Treasury at the specified address\n- FOB Point is Destination\n- Include completed copy of provision at 52.212-3, Offeror Representations and Certifications-Commercial Products and Commercial Services, with offer.',
 '- 1:1 Coaching by ICF credentialed coaches\n- On-Demand Coaching with sessions occurring within one business day\n- Learning/Development Resources to support coachees\n- Assessment to be completed at start and end of engagements',
 "- Development of a coaching plan within the first two sessions\n- Unlimited coaching sessions for the term of individual license\n- Coaches must be International Coach Federation credentialed\n- At least one 360 Assessment provided on the platform must align with Office of Personnel Management's Executive Core Qualifications (ECQs) and include reports for the coachee",
 '- Complet

In [20]:
len(new_requirmeents)

285